In [5]:
using LightGraphs
using GraphIO

function simulateLin(G, init_nodes)
	
	num_edges = ne(G)
	num_vertices = nv(G)
	active = copy(init_nodes)
    
	#generate the tresholds

	tsh = rand(num_vertices)

	#initialize weight array

	weights = ones(num_edges)

	#degree array

	deg = degree(G)

	#set weightsprintln(sum(resLin))

	for (i, e) in enumerate(edges(G))
		weights[i] = 1/deg[dst(e)]
	end
	
	something_happened = true

	while(something_happened)
		something_happened = false
        
        #we use an array to keep track, for each node, the sum of the weights of all active neighbors
        
		sums = zeros(num_vertices)
		
        #we iterate over all edges and add the sum of the weights of all active neighbors
        
		for (i, e) in enumerate(edges(G))
			if active[src(e)] == 1
				sums[dst(e)] = sums[dst(e)] + weights[i]
			end
		end
        		
        #we then iterate over all nodes and activate the ones where the sum surpasses the treshold
        
		for v in vertices(G)
			if(active[v] == 0 && sums[v] > tsh[v])
				active[v] = 1
                something_happened = true
			end
		end
    end
	return active
	
end

function simulateCasc(G, init_nodes)

	num_edges = ne(G)
	num_vertices = nv(G)
	active = copy(init_nodes)

	prob = rand(num_vertices)
	
	#vector that indicates which nodes just became active. 0 = never was active ; 1 = just became active ; 2 = already was active
	
	just_became_active = copy(init_nodes)
	
	something_happened = true
	
	while(something_happened)
		something_happened = false
		
		#iterate over all nodes that just became active
		
		for (v, state) in enumerate(just_became_active)
			if state == 1
				
				#if an active node was considered, set its state to 2 so it doesn't get considered again
				
				just_became_active[v] = 2
				
				#we iterate over all neighbors of a node that just became active
				
				for n in neighbors(G, v)
					
					#if the neighbor is not active
					
					if(active[n] == 0)
						
						#toss a coin wether he becomes active
						
						coin = rand()
						
						if coin > prob[n]
							
							#if it becomes active, set both as active and just became active
							
							active[n] = 1
							just_became_active[n] = 1
							something_happened = true
						end
					end
				end
			end			
		end
	end
	
	return active
	
end

function greedyLin(G, giveaway_number, num_sim)
    println("Starting linear greedy algorithm ...")
    
	num_sim_real = 0
	num_edges = ne(G)
	num_vertices = nv(G)
	
	active = zeros(num_vertices)
	
	for i in range(1, giveaway_number)
		high_score = 0
		high_score_edge = 0
		
		for e in vertices(G)
			if(active[e] == 0)
				init_nodes = copy(active)
				init_nodes[e] = 1
				score = 0
				
				for j in range(1,num_sim)
					score = score + sum(simulateLin(G, init_nodes))
					num_sim_real = num_sim_real +1
				end
				
				if(score > high_score)
					high_score = score
					high_score_edge = e
				end
			end
		end
		
		active[high_score_edge] = 1
        
        println("k = ", i, " ; numSim = ", num_sim, " ; avg activations = ", high_score/num_sim)
		
	end
    
    return active
	
end

function greedyCasc(G, giveaway_number, num_sim)
    println("Starting cascade greedy algorithm ...")

	num_sim_real = 0
	num_edges = ne(G)
	num_vertices = nv(G)
	
	active = zeros(num_vertices)
	
	for i in range(1, giveaway_number)
		high_score = 0
		high_score_edge = 0
		
		for e in vertices(G)
			if(active[e] == 0)
				init_nodes = copy(active)
				init_nodes[e] = 1
				score = 0
				
				for j in range(1,num_sim)
					score = score + sum(simulateCasc(G, init_nodes))
					num_sim_real = num_sim_real +1
				end
				
				if(score > high_score)
					high_score = score
					high_score_edge = e
				end
			end
		end
		
		active[high_score_edge] = 1
        println("k = ", i, " ; numSim = ", num_sim, " ; avg activations = ", high_score/num_sim)

	end
    
	return active
	
end
					
	
G = loadgraph("/home/chris/Uni/GraphMining/Programming/Progetto/Graph/3980.edges", "facebook", EdgeListFormat())


res = greedyCasc(G,50,100000)



Starting cascade greedy algorithm ...
k = 1 ; numSim = 100000 ; avg activations = 43.99995
k = 2 ; numSim = 100000 ; avg activations = 47.99984
k = 3 ; numSim = 100000 ; avg activations = 49.99993
k = 4 ; numSim = 100000 ; avg activations = 51.99985
k = 5 ; numSim = 100000 ; avg activations = 51.99989
k = 6 ; numSim = 100000 ; avg activations = 51.99993
k = 7 ; numSim = 100000 ; avg activations = 51.99993
k = 8 ; numSim = 100000 ; avg activations = 51.99995
k = 9 ; numSim = 100000 ; avg activations = 51.99995
k = 10 ; numSim = 100000 ; avg activations = 51.99995
k = 11 ; numSim = 100000 ; avg activations = 51.99995
k = 12 ; numSim = 100000 ; avg activations = 51.99995
k = 13 ; numSim = 100000 ; avg activations = 51.99995
k = 14 ; numSim = 100000 ; avg activations = 51.99992
k = 15 ; numSim = 100000 ; avg activations = 51.99993
k = 16 ; numSim = 100000 ; avg activations = 51.99995
k = 17 ; numSim = 100000 ; avg activations = 51.99999
k = 18 ; numSim = 100000 ; avg activations = 51.99997

52-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [15]:
res = greedyLin(G,2,100000)


Starting linear greedy algorithm ...
k = 1 ; numSim = 100000 ; avg activations = 43.99796


LoadError: [91mInterruptException:[39m